# Train PointNet (https://arxiv.org/abs/1612.00593).

This notebook shows you how to use the PreprocessedDataGenerator in order to train PointNet.

The PreprocessedDataGenerator uses preprocessed-data instead of ETL-data. Wheras ETL-data comes mainly as PCD-files, preprocessed-data comes mainly as pointclouds stored as numpy-arrays. We identified PCD-loading as a bottleneck. 

In [1]:
import sys
sys.path.insert(0, "..")

import numpy as np
import os
import random

# Get the dataset path.

This snippet shows you how to get the lates preprocessed path.

In [2]:
from cgmcore.preprocesseddatagenerator import get_dataset_path

dataset_path = get_dataset_path("../../data/preprocessed")
print("Using dataset path", dataset_path)

WARNING! VTK not available. This might limit the functionality.
Using dataset path ../../data/preprocessed/2018_10_31_14_19_42


# Hyperparameters.

In [3]:
steps_per_epoch = 10
validation_steps = 10
epochs = 4
batch_size = 1
random_seed = 667

# Create data-generator.

The method create_datagenerator_from_parameters is a convencience method. It allows you to instantiate a generator from a specification-dictionary.

In [4]:
from cgmcore.preprocesseddatagenerator import create_datagenerator_from_parameters

dataset_parameters_pointclouds = {}
dataset_parameters_pointclouds["input_type"] = "pointcloud"
dataset_parameters_pointclouds["output_targets"] = ["height"]
dataset_parameters_pointclouds["random_seed"] = random_seed
dataset_parameters_pointclouds["pointcloud_target_size"] = 10000
dataset_parameters_pointclouds["pointcloud_random_rotation"] = False
dataset_parameters_pointclouds["sequence_length"] = 0
datagenerator_instance_pointclouds = create_datagenerator_from_parameters(dataset_path, dataset_parameters_pointclouds)

Creating data-generator...


# Getting the QR-Codes and do a train-validate-split.

The data-generator is perfectly capable of retrieving all QR-codes from the dataset. This snipped shows how to do so and how to split the QR-codes into two sets: Train and validate.

In [5]:
# Get the QR-codes.
qrcodes_to_use = datagenerator_instance_pointclouds.qrcodes[0:30]

# Do the split.
random.seed(random_seed)
qrcodes_shuffle = qrcodes_to_use[:]
random.shuffle(qrcodes_shuffle)
split_index = int(0.8 * len(qrcodes_shuffle))
qrcodes_train = sorted(qrcodes_shuffle[:split_index])
qrcodes_validate = sorted(qrcodes_shuffle[split_index:])
del qrcodes_shuffle
print("QR-codes for training:\n", "\t".join(qrcodes_train))
print("QR-codes for validation:\n", "\t".join(qrcodes_validate))

QR-codes for training:
 MH_WHH_0001	MH_WHH_0003	MH_WHH_0004	MH_WHH_0009	MH_WHH_0010	MH_WHH_0014	MH_WHH_0016	MH_WHH_0017	MH_WHH_0019	MH_WHH_0022	MH_WHH_0028	MH_WHH_0036	MH_WHH_0044	MH_WHH_0054	MH_WHH_0063	MH_WHH_0075	MH_WHH_0076	MH_WHH_0081	MH_WHH_0082	MH_WHH_0083	MH_WHH_0095	MH_WHH_0096	MH_WHH_0102	MH_WHH_0104
QR-codes for validation:
 MH_WHH_0008	MH_WHH_0027	MH_WHH_0030	MH_WHH_0056	MH_WHH_0077	MH_WHH_0097


# Creating python generators for training and validation.

Now both QR-codes lists can be used for creating the actual generators. One for training and one for validation.

In [6]:
# Create python generators.
generator_pointclouds_train = datagenerator_instance_pointclouds.generate(size=batch_size, qrcodes_to_use=qrcodes_train)
generator_pointclouds_validate = datagenerator_instance_pointclouds.generate(size=batch_size, qrcodes_to_use=qrcodes_validate)

# Using the generator to create data manually.

Of course you can use the generator to create data manually anytime.

In [12]:
train_x, train_y = next(generator_pointclouds_train)
print("Input-shape:", train_x.shape)
print("Output-shape:", train_y.shape)

Input-shape: (1, 10000, 3)
Output-shape: (1, 1)


# Training-details.

Training-details are a dictionary that gets stored in a file after training. It is supposed to contain information that is valuable. For example data that is relevant for training including the hyper-parameters. Intended to be used when comparing different models.

In [7]:
training_details = {
    "dataset_path" : dataset_path,
    "qrcodes_train" : qrcodes_train,
    "qrcodes_validate" : qrcodes_validate,
    "steps_per_epoch" : steps_per_epoch,
    "validation_steps" : validation_steps,
    "epochs" : epochs,
    "batch_size" : batch_size,
    "random_seed" : random_seed,
}

# Training PointNet.

The module modelutils contains methods for creating Neural Nets. The following code shows how to instantiate and train PointNet.

In [8]:
from cgmcore import modelutils

input_shape = (dataset_parameters_pointclouds["pointcloud_target_size"], 3)
output_size = 1
model_pointnet = modelutils.create_point_net(input_shape, output_size, hidden_sizes = [64])
model_pointnet.summary()
    
model_pointnet.compile(
    optimizer="rmsprop",
    loss="mse",
    metrics=["mae"]
    )

history = model_pointnet.fit_generator(
    generator_pointclouds_train,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=generator_pointclouds_validate,
    validation_steps=validation_steps
    )

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10000, 3)          0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 10000, 3)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 10000, 64)         256       
_________________________________________________________________
batch_normalization_6 (Batch (None, 10000, 64)         256       
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 10000, 64)         4160      
_________________________________________________________________
batch_normalization_7 (Batch (None, 10000, 64)         256       
_________________________________________________________________
lambda_2 (Lambda)            (None, 10000, 64)         0         
__________

# Saving everything.

This saves the model, its history and the training-details to some output directory. The created artifacts can later be uses in order to compare different models.

In [9]:
output_path = "."

modelutils.save_model_and_history(output_path, model_pointnet, history, training_details, "pointnet")

Saving model and history...
WARNING! Failed to save model. Use model-weights instead.
Saved model weights to./20181102-1208-pointnet-model-weights.h5
Saved training details to./20181102-1208-pointnet-details.p
Saved history to./20181102-1208-pointnet-history.p
